# Extract relevant genes and generate files in intermediate_data_01/

## Workflow description

1. Run **I. Essentials** and **II. Custom Functions**
2. For each dataset in `lipid_selection/data/raw_data/source_data/`:

    1. Extract basic information:
        - genome_version 
        - database_source 
        - inclusion_criteria 
        - first_author
        - publication_year
      
    2. Append basic information to `lipid_selection/data/intermediate_data_01/basic_info.txt`
        - Use `append_basic_info()`
    
    3. Extract candidate and non-candidate genes.
        - Use `check_excel_data()`, `import_messy_excel()`
    
    4. Export candidate and non-candidate genes to `<first_author>_<year>.txt`.
        - Use `export_genes()` or `export_proteins()`

## I. Essentials

#### Check directories and load library packages

Working directory is `lipid_selection/data/raw_data/source_data`.

In [1]:
current_dir = getwd()
source_data_dir = "../../data/raw_data/source_data"
setwd(source_data_dir)

#Set target folder for candidate gene info from step 2.D
target_folder = "../../intermediate_data_01/"

Load essential library packages

In [2]:
library("readxl")
library("dplyr")
library("tidyverse")


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.1.1     v readr   1.3.1
v tibble  2.1.1     v purrr   0.3.2
v tidyr   0.8.3     v stringr 1.4.0
v ggplot2 3.1.1     v forcats 0.4.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


In [3]:
getwd()

[1] "/home/chenwe72/gitRepo/lipid_selection/data/raw_data/source_data"

## II. Custom functions

* `check_excel_data`
    - **Usage**: Check if excel dataset has more than one sheet; check if file type is .xls or .xlsx
    - **input**: file_path (str)
    - **output**: list of sheet names as strings
    
    
* `import_messy_excel`
    - **Usage**: Remove non-data table rows and import a cleaner dataframe from an excel dataset
    - **input**: file_path (str), sheet_name (str)
    - **output**: dataframe
    
    
* `export_genes`
    - **Usage**: Export candidate and non-candidate genes to `<first_author>_<year>.txt` in `target_folder`
    - **input**: gene_data (dataframe), 
    - **output**: `<first_author>_<year>.txt` in `target_folder`
    
    - **Usage**: 
    - **input**: 
    - **output**: 

#### check_excel_data

In [4]:
#Check if excel dataset has more than one sheet. 
#Check if file type is .xls or .xlsx
check_excel_data <- function (file_path){
    
    #If file type is not .xls or .xlsx, return FALSE
    #Return sheet name(s) as a list of strings
    
    #check if "readxl" is loaded
    require("readxl") 
    
    #check if file type is .xls or .xlsx. excel_sheets() only works with these file types
    if (strsplit(file_path, "[.]")[[1]][2] %in% c("xls", "xlsx")){
        list_of_sheets <- excel_sheets(file_path)
        return (list_of_sheets)
    }
    else {
        stop("File type is not .xls or .xlsx")} 
    
}

In [5]:
#Test code with excel dataset with more than one sheet
check_excel_data("Bajhaiya_2016.xls")  

#Test code with excel dataset with one sheet
check_excel_data("Boyle_2012.xls") 

#Test code with incorrect file type
check_excel_data("Li_2016.xlsb")

[1] "Day 3" "Day 5"

[1] "Sheet1"

ERROR: Error in check_excel_data("Li_2016.xlsb"): File type is not .xls or .xlsx


#### import_messy_excel

In [6]:
#Remove non-data table rows and import a cleaner dataframe from an excel dataset

import_messy_excel <- function(file_path, sheet_name){
    
    #check if "readxl" is loaded
    require("readxl") 
    require("dplyr")
    require("stringr")
    
    #Checks to see if sheet name exists
    if (sheet_name %in% excel_sheets(file_path) == FALSE){
            stop("Sheet name does not exist")
    }
    
    #Remove rows in excel sheet if more than half of the columns have NAs
    df <- read_excel(file_path, sheet = sheet_name) %>% 
        filter(rowSums(is.na(.))/ncol(.) < 0.5)
    
    #Assumes Row 1 of the subset dataframe is the column names
    #Assign Row 1 as column names and remove Row 1
    colnames(df)<- df[1,] %>% str_replace_all(" ", "_")
    df<- df[-1,]
    message("Check if column names are correct.")
    
    return(df)
    
}

#### export_genes

`gene_data` should have the some of the following columns *(when available)* :
- `gene_id` : <`string`> matches `^Cre.+`
- `gene_name` : <`string`> abbreviation of gene name 
- `protein_id`: <`string`> 
- `candidate_gene` : <`boolean`> whether the gene is candidate or not
- `fold_difference`: <`numeric`> linear fold-difference in gene expression levels
- `p_value`: <`numeric`> p-value of whether `|log2(fold_difference)| >1`
- `protein_fold_difference`: <`numeric`> linear fold-difference in protein expression levels
- `protein_p_value`: <`numeric`> p-value of whether `|log2(protein_fold_difference)| >1`


In [258]:
#Export candidate and non-candidate genes to `<first_author>_<year>.txt` in target_folder
#Input: gene_data as dataframe

export_data <- function(df = gene_data, target_folder = target_folder){
    
    require("dplyr")
       
    write.table(gene_data, 
            paste(target_folder, first_author,"_",publication_year,".txt", sep=""), 
            quote = FALSE, sep = "\t", col.names = TRUE, row.names = FALSE)
    
}

#### append_basic_info

In [8]:
#Input basic_info.txt as output_filepath
#basic_info is a dataframe

append_basic_info <- function(basic_info = basic_info, output_filepath){
    
    colnames(basic_info)<- NULL
    #Append basic info to output_filepath
    write.table(basic_info, output_filepath, 
                append = TRUE, sep = "\t", quote = FALSE, 
                row.names = FALSE)
    
}

### Create basic_info.txt in intermediate_data_01

In [9]:
basic_info <- data.frame(matrix(ncol = 5, nrow = 0))
colnames(basic_info) <- c("first_author",
                         "publication_year",
                         "genome_version",
                         "database_source", 
                         "inclusion_criteria")
write.table(basic_info, "../../intermediate_data_01/basic_info.txt", 
            sep = "\t", quote = FALSE, row.names = FALSE)

## II. Add source data to intermediate_data_01

### Bajhaiya_2016.xls

#### 1. Extract basic information
#### 2. Append basic information to `lipid_selection/data/intermediate_data_01/basic_info.txt`

In [149]:
#Basic information

file_name = "Bajhaiya_2016.xls"
genome_version = 5.3
database_source = "Phytozome 9.1"
inclusion_criteria = "fold difference greater than 2"
first_author = "Bajhaiya"
publication_year = 2016

basic_info <- data.frame(first_author, 
                         publication_year, 
                         genome_version, 
                         database_source, 
                         inclusion_criteria, 
                         stringsAsFactors = FALSE)
str(basic_info)

append_basic_info(basic_info, "../../intermediate_data_01/basic_info.txt")

'data.frame':	1 obs. of  5 variables:
 $ first_author      : chr "Bajhaiya"
 $ publication_year  : num 2016
 $ genome_version    : num 5.3
 $ database_source   : chr "Phytozome 9.1"
 $ inclusion_criteria: chr "fold difference greater than 2"


In [150]:
#Check file type and number of excel sheets
sheets <- check_excel_data(file_name)  
print(length(sheets))

[1] 2


#### 3. Extract candidate and non-candidate genes based on inclusion criteria.

**Inclusion criteria**: Within each strain, if fold-difference between high P and low P is >2 count as candidate gene. 

**Sheet 1: "Day 3" **

In [185]:
df <- import_messy_excel(file_name, sheets[1]) 
#Use fold difference from normalized expression
temp1 <- df %>% 
    select(starts_with("Gene"), ends_with("foldchange")) %>%
    mutate_at(vars(ends_with("foldchange")),list(as.numeric)) %>%

    #reshape data such that one of the columns is fold difference
    gather(., 'WT_LP_/_HP_D3_foldchange','psr1_LP_/_HP_D3_foldchange', 
           key = 'treatment', value = 'fold_difference') %>%

    #assign candidate gene label
    mutate(candidate_gene = case_when(
        abs(log(fold_difference,2))>1 ~ TRUE,
        TRUE ~ FALSE)) %>%
    
    #select columns that go in to working gene_data dataframe
    select(starts_with("Gene"), candidate_gene, fold_difference)
           
temp1[1:3,]

New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* ... and 13 more problems
Check if column names are correct.


Gene_id,Gene_name,candidate_gene,fold_difference
<chr>,<chr>,<lgl>,<dbl>
Cre09.g404900,Cre09.g404900,TRUE,85.99029
Cre04.g216700,PHOX,TRUE,82.75193
Cre01.g044300,Cre01.g044300,TRUE,71.10010


** Sheet 2: "Day 5" **

In [186]:
df <- import_messy_excel(file_name, sheets[2]) 
#Use fold difference from normalized expression
temp2 <- df %>% 
    select(starts_with("Gene"), ends_with("foldchange")) %>%
    mutate_at(vars(ends_with("foldchange")),list(as.numeric)) %>%

    #reshape data such that one of the columns is fold difference
    gather(., 'WT_LP_/_HP_D5_foldchange','psr1_LP_/_HP_D5_foldchange', 
           key = 'treatment', value = 'fold_difference') %>%

    #assign candidate gene label
    mutate(candidate_gene = case_when(
        abs(log(fold_difference,2))>1 ~ TRUE,
        TRUE ~ FALSE)) %>%
    
    #select columns that go into working gene_data dataframe
    select(starts_with("Gene"), candidate_gene, fold_difference)
           
temp2[1:3,]

New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* ... and 13 more problems
Check if column names are correct.


Gene_id,Gene_name,candidate_gene,fold_difference
<chr>,<chr>,<lgl>,<dbl>
Cre09.g404900,Cre09.g404900,TRUE,15.11795
Cre04.g216700,PHOX,TRUE,13.01713
Cre01.g044300,Cre01.g044300,TRUE,18.13303


##### Join Sheet 1 and 2 data by creating two dataframes:

In [187]:
gene_data <- rbind(temp1, temp2) %>%
    rename(gene_id = Gene_id,
          gene_name = Gene_name)

gene_data[1:3,]
#print(summary(gene_data))

# A tibble: 3 x 4
  gene_id       gene_name     candidate_gene fold_difference
  <chr>         <chr>         <lgl>                    <dbl>
1 Cre09.g404900 Cre09.g404900 TRUE                      86.0
2 Cre04.g216700 PHOX          TRUE                      82.8
3 Cre01.g044300 Cre01.g044300 TRUE                      71.1
   gene_id           gene_name         candidate_gene  fold_difference   
 Length:70948       Length:70948       Mode :logical   Min.   :  0.0019  
 Class :character   Class :character   FALSE:60442     1st Qu.:  0.8118  
 Mode  :character   Mode  :character   TRUE :10506     Median :  1.0000  
                                                       Mean   :  1.2409  
                                                       3rd Qu.:  1.2824  
                                                       Max.   :417.8523  


#### 4. Export `gene_data` to `Bajhaiya_2016.txt`.

In [189]:
temp<- export_genes(df = gene_data, target_folder = target_folder)
temp[1:5,]
dim(temp)

gene_id,gene_name,candidate_gene,fold_difference,protein_id,protein_name
<chr>,<chr>,<lgl>,<dbl>,<lgl>,<lgl>
Cre09.g404900,Cre09.g404900,TRUE,85.99029,NA,NA
Cre04.g216700,PHOX,TRUE,82.75193,NA,NA
Cre01.g044300,Cre01.g044300,TRUE,71.10010,NA,NA
g16424,g16424,TRUE,58.16937,NA,NA
g2975,MPA1,TRUE,51.21199,NA,NA


[1] 70948     6

## Blaby_2013_DS2.xlsx

#### 1. Extract basic information
#### 2. Append basic information to `lipid_selection/data/intermediate_data_01/basic_info.txt`

In [259]:
#Basic information

file_name = c("Blaby_2013_DS2.xlsx","Blaby_2013_DS8.xlsx")
genome_version = 5.0
database_source = "Augustus u10.2"
inclusion_criteria = "fold difference greater than 2"
first_author = "Blaby"
publication_year = 2013

basic_info <- data.frame(first_author, 
                         publication_year, 
                         genome_version, 
                         database_source, 
                         inclusion_criteria, 
                         stringsAsFactors = FALSE)
str(basic_info)

append_basic_info(basic_info, "../../intermediate_data_01/basic_info.txt")

'data.frame':	1 obs. of  5 variables:
 $ first_author      : chr "Blaby"
 $ publication_year  : num 2013
 $ genome_version    : num 5
 $ database_source   : chr "Augustus u10.2"
 $ inclusion_criteria: chr "fold difference greater than 2"


In [260]:
#Check file type and number of excel sheets
sheets <- check_excel_data(file_name[1])  
print(length(sheets))
sheets <- check_excel_data(file_name[2])  
print(length(sheets))

[1] 1
[1] 1


#### 3. Extract candidate and non-candidate genes based on inclusion criteria.

**Inclusion criteria**: Within each strain, if fold-difference between 0 hour and *n* hours after N starvation >2 count as candidate gene. 

* Gene expression differences due to *sta-6* mutation is less important than gene expression differences due to nutrient starvation

In [266]:
require("readxl")
require("tidyverse")
require("dplyr")

#Data manipulation to separate data by strains
CC_4349 <- data.frame(matrix(nrow = 0, ncol = 11))
sta_6 <-data.frame(matrix(nrow = 0, ncol = 11))
for (file in file_name){
    df<- import_messy_excel(file, sheets[1])
    CC_4349 <- df[,1:11] %>% 
        mutate(strain = "CC_4349") %>% 
        rename ( '0' = '0b') %>% 
        rbind(CC_4349,.)
    sta_6 <- df[, c(1:3, 12:19)] %>% mutate(strain = "sta_6") %>%
        rbind(CC_4349,.)
}
 
 
CC_4349[1:3,]

sta_6[1:3,]

New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* ... and 13 more problems
Check if column names are correct.
New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* ... and 13 more problems
Check if column names are correct.


Gene,Annotation,Augustus_u10.2_ID,0,0.5,2,4,8,12,24,48,strain
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
NA,NA,Cre01.g001100,41.863999999999997,45.618200,30.0444200,28.7763500,20.1547200,20.2106500,20.2113900,23.325140,CC_4349
NA,NA,Cre01.g004750,0.1167516,0.232438,0.1756702,0.1662885,0.1605637,0.2873106,0.7915955,1.283815,CC_4349
FBA1,"Fructose-1,6-bisphosphate aldolase",Cre01.g006950,98.351749999999996,155.951700,117.9667000,57.0684500,27.8726400,23.5809200,28.8262100,24.370270,CC_4349


Gene,Annotation,Augustus_u10.2_ID,0,0.5,2,4,8,12,24,48,strain
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
NA,NA,Cre01.g001100,41.863999999999997,45.618200,30.0444200,28.7763500,20.1547200,20.2106500,20.2113900,23.325140,CC_4349
NA,NA,Cre01.g004750,0.1167516,0.232438,0.1756702,0.1662885,0.1605637,0.2873106,0.7915955,1.283815,CC_4349
FBA1,"Fructose-1,6-bisphosphate aldolase",Cre01.g006950,98.351749999999996,155.951700,117.9667000,57.0684500,27.8726400,23.5809200,28.8262100,24.370270,CC_4349


In [267]:
gene_data <- rbind(CC_4349, sta_6) %>% 

    #reshape dataframe to assign candidate_gene label based on fold difference
    gather(.,'0.5', '2', '4', '8', '12', '24', '48', 
      key = "time", value = 'expression') %>%
    rename(time_0h = '0') %>%
    mutate(time_0h = as.numeric(time_0h)) %>%

    #assign candidate gene label based on fold difference
    mutate( candidate_gene = case_when(
        expression >= 2*time_0h ~ TRUE,
        time_0h>= 2*expression ~ TRUE,
        TRUE ~ FALSE),
    #calculate fold difference relative to time 0H control
          fold_difference = time_0h/expression) %>%

    #select columns that go into working gene_data dataframe
    rename (gene_name = Gene, gene_id = Augustus_u10.2_ID) %>%
    select( gene_id, gene_name, candidate_gene, fold_difference)

#### 4. Export `gene_data` to `Blaby_2013.txt`

In [269]:
gene_data[1:3,]
temp<- export_genes(df = gene_data, target_folder = target_folder)
dim(temp)

gene_id,gene_name,candidate_gene,fold_difference
<chr>,<chr>,<lgl>,<dbl>
Cre01.g001100,NA,FALSE,0.9177039
Cre01.g004750,NA,FALSE,0.5022914
Cre01.g006950,FBA1,FALSE,0.6306552


[1] 10395     6

### Boyle_2012.xls

#### 1. Extract basic information
#### 2. Append basic information to `lipid_selection/data/intermediate_data_01/basic_info.txt`

In [223]:
file_name = "Boyle_2012.xls"
genome_version = 4.0
database_source = "Augustus 10.2"
inclusion_criteria = "fold difference greater than 2"
first_author = "Boyle"
publication_year = 2012

basic_info <- data.frame(first_author, 
                         publication_year, 
                         genome_version, 
                         database_source, 
                         inclusion_criteria, 
                         stringsAsFactors = FALSE)
str(basic_info)

append_basic_info(basic_info, "../../intermediate_data_01/basic_info.txt")

'data.frame':	1 obs. of  5 variables:
 $ first_author      : chr "Boyle"
 $ publication_year  : num 2012
 $ genome_version    : num 4
 $ database_source   : chr "Augustus 10.2"
 $ inclusion_criteria: chr "fold difference greater than 2"


In [224]:
#Check file type and number of excel sheets
sheets <- check_excel_data(file_name)  
print(length(sheets))

[1] 1


#### 3. Extract candidate and non-candidate genes based on inclusion criteria.

**Inclusion criteria**: If fold-difference of RPKM between 0 hour and *n* hours after N starvation >2 count as candidate gene. 

In [225]:
require("readxl")
require("tidyverse")
require("dplyr")

#Import excel sheet and rename column names
df <- import_messy_excel(file_name, sheets[1])
colnames(df)[1:5]<- c("gene_id", "Au.5", "gene_name","protein_name", "time_0h")
df[1:3,]

New names:
* `` -> ...6
* `` -> ...7
* `` -> ...8
* `` -> ...9
Check if column names are correct.


gene_id,Au.5,gene_name,protein_name,time_0h,2_h,12_h,24_h,48_h
<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Cre12.g519100,513248,ACX1,?-Carboxyltransferase,140.5,68.200000000000003,81.299999999999997,71.599999999999994,70.200000000000003
Cre12.g484000,512497,BCX1,?-Carboxyltransferase,103.5,53.399999999999999,35.100000000000001,35.399999999999999,31.399999999999999
Cre17.g715250,517403,BCC1,Acetyl-CoA biotin carboxyl carrier,293.89999999999998,129.59999999999999,54.700000000000003,74.700000000000003,68.900000000000006


In [200]:
#Reshape dataframe such that it is easier to compare time 0h to 'n'h
gene_data<- df %>% gather(., '2_h', '12_h', "24_h", '48_h',
                   key = 'time', value = "expression") %>%
    mutate(time_0h = as.numeric(time_0h),
        expression= as.numeric(expression)) %>%
           
    #assign candidate_gene label based on fold difference
    mutate(candidate_gene = case_when(
            expression >= 2*time_0h~ TRUE,
            time_0h >= 2* expression~ TRUE,
            TRUE ~ FALSE),
          fold_difference = time_0h/expression) %>%

    #aggregate dataframe and remove unnecessary columns for append_genes()
    select(gene_id, gene_name, candidate_gene, fold_difference)

#gene_data[1:3,]
#summary(gene_data)

gene_id,gene_name,candidate_gene,fold_difference
<chr>,<chr>,<lgl>,<dbl>
Cre12.g519100,ACX1,TRUE,2.060117
Cre12.g484000,BCX1,FALSE,1.938202
Cre17.g715250,BCC1,TRUE,2.267747


   gene_id           gene_name         candidate_gene  fold_difference 
 Length:100         Length:100         Mode :logical   Min.   :0.0000  
 Class :character   Class :character   FALSE:49        1st Qu.:0.8473  
 Mode  :character   Mode  :character   TRUE :51        Median :1.3939  
                                                       Mean   :   Inf  
                                                       3rd Qu.:3.0234  
                                                       Max.   :   Inf  
                                                       NA's   :3       

gene_id,gene_name,candidate_gene,fold_difference,protein_id,protein_name
<chr>,<chr>,<lgl>,<dbl>,<lgl>,<lgl>
Cre12.g519100,ACX1,TRUE,2.060117,NA,NA
Cre12.g484000,BCX1,FALSE,1.938202,NA,NA
Cre17.g715250,BCC1,TRUE,2.267747,NA,NA


#### 4. Export `gene_data` to `Boyle_2012.txt`.

In [ ]:
temp<-export_genes(df = gene_data, target_folder = target_folder)
temp[1:3,]

## Gargouri_2015.xlsx
#### 1. Extract basic information
#### 2. Append basic information to `lipid_selection/data/intermediate_data_01/basic_info.txt`

In [229]:
#Basic information

file_name = "Gargouri_2015.xlsx"
genome_version = NA
database_source = "Phytozyme 10.0"
inclusion_criteria = c("fold difference greater than 2", "p-value <0.05")
first_author = "Gargouri"
publication_year = 2015

basic_info <- data.frame(first_author, 
                         publication_year, 
                         genome_version, 
                         database_source, 
                         inclusion_criteria, 
                         stringsAsFactors = FALSE)
str(basic_info)

append_basic_info(basic_info, "../../intermediate_data_01/basic_info.txt")

'data.frame':	2 obs. of  5 variables:
 $ first_author      : chr  "Gargouri" "Gargouri"
 $ publication_year  : num  2015 2015
 $ genome_version    : logi  NA NA
 $ database_source   : chr  "Phytozyme 10.0" "Phytozyme 10.0"
 $ inclusion_criteria: chr  "fold difference greater than 2" "p-value <0.05"


In [230]:
#Check file type and number of excel sheets
sheets <- check_excel_data(file_name)  
print(length(sheets))
print(sheets)

[1] 16
 [1] "TF Gene Expression"     "TR Gene Expression "    "TF & TR proteins"      
 [4] "R values"               "P values"               "Correlation lists"     
 [7] "Nitrogen genes"         "Photosynthesis genes"   "Chlorophyll genes"     
[10] "Calvin cycle genes"     "Photorespiration genes" "OPPP genes"            
[13] "TCA-glyoxylate genes"   "Amino acid genes"       "sucrose-starch genes"  
[16] "Lipid genes"           


#### 3. Extract candidate and non-candidate genes based on inclusion criteria.

**Inclusion criteria**: When fold-difference between 0 hour and *n* hours after N starvation is statistically signficant (false discovery rate adjusted p-value <0.05), count as candidate gene. 
    
* Significance in expression is defined as >2 fold-difference in a time point relative to the control (time= 0 hours)
* Exclude proteomics information because the study measured transcription factor and regulator proteins. These proteins have low copy number in the cells because they don't need to exist at higher copy numbers. 

In [231]:
require("readxl")
require("tidyverse")
require("dplyr")

#Data manipulation: combine TF and TR gene expression data
df <- rbind(import_messy_excel(file_name, sheet_name = "TF Gene Expression") , import_messy_excel(file_name, sheet_name = "TR Gene Expression "))
colnames(df)[1:3] <- c("gene_id","", "gene_name")

#Split data into p-value data and fold difference data
p_value_df<-df[,c(1,3,18:24)]
log2fold_diff_df<-df[,c(1,3,4:10)]
p_value_df[1:3,]
log2fold_diff_df[1:3,] 

New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* ... and 18 more problems
Check if column names are correct.
New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* ... and 18 more problems
Check if column names are correct.


gene_id,gene_name,0.5h,1h,2h,4h,6h,12h,24h
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Cre13.g562400,ABI3,0.26624138637167877,0.3784149427406196,0.17107280180360585,8.9693481346717077E-2,0.67470075346544178,0.19118399314252876,9.9176569946903728E-2
Cre16.g661650,AP2.1,0.38964820106104703,0.60777284815719523,0.81484172824510082,0.24174437138377752,0.68594084635814501,0.31886254595637914,0.16692534218800126
Cre06.g275500,AP2.2,0.32259173012359688,0.28657095682324008,0.97631043910190984,0.91747186557718341,0.86060237923868088,0.35019419753972997,0.13989581549811209


gene_id,gene_name,0.5h,1h,2h,4h,6h,12h,24h
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Cre13.g562400,ABI3,0.24499975863038514,0.15281513303578501,0.65829909409954446,0.5963349885218836,0.83775795243780415,0.49261075369497898,0.829104318114255
Cre16.g661650,AP2.1,-0.2049811425946774,-3.73423144159995E-2,0.27253941891884886,-0.3849485106593602,-0.42471729292313487,-0.41541845462348742,-0.52049019071806291
Cre06.g275500,AP2.2,0.54495196817653324,-0.99111493744001344,-0.13882746250235672,-0.2196249599012253,4.1920206177396412E-2,0.33344408720170204,0.52174949951430005


In [232]:
#Reshape dataframe such that it is easier to compare time 0h to 'n'h
p_value_df <- p_value_df %>% 
    gather(., ends_with('h'),
           key = 'time', value = "p_value") %>%
    mutate(p_value= as.numeric(p_value))
log2fold_diff_df <- log2fold_diff_df  %>% 
    gather(., ends_with('h'),
           key = 'time', value = "log2fold_diff") %>%
    mutate(log2fold_diff= as.numeric(log2fold_diff))
gene_data<- merge(log2fold_diff_df, p_value_df, 
                  by = c('gene_id', 'gene_name', 'time')) %>%
            mutate(candidate_gene = case_when(
                p_value< 0.05 ~TRUE,
                TRUE~FALSE),
                  fold_difference = 2^log2fold_diff) %>%
            select(gene_id, gene_name, candidate_gene,fold_difference, p_value)


gene_data[1:3,]
#summary(gene_data)


gene_id,gene_name,candidate_gene,fold_difference,p_value
<chr>,<chr>,<lgl>,<dbl>,<dbl>
Cre01.g000050,RWP.14,FALSE,1.5629104,0.0912037
Cre01.g000050,RWP.14,FALSE,1.9702945,0.3410131
Cre01.g000050,RWP.14,FALSE,0.9607626,0.4324472


   gene_id           gene_name         candidate_gene  fold_difference  
 Length:2898        Length:2898        Mode :logical   Min.   : 0.1083  
 Class :character   Class :character   FALSE:2883      1st Qu.: 0.8376  
 Mode  :character   Mode  :character   TRUE :15        Median : 1.1439  
                                                       Mean   : 1.5107  
                                                       3rd Qu.: 1.5859  
                                                       Max.   :40.0687  
    p_value       
 Min.   :0.03828  
 1st Qu.:0.19014  
 Median :0.36772  
 Mean   :0.42771  
 3rd Qu.:0.66466  
 Max.   :1.00000  

gene_id,gene_name,candidate_gene,fold_difference,p_value,protein_id,protein_name
<chr>,<chr>,<lgl>,<dbl>,<dbl>,<lgl>,<lgl>
Cre01.g000050,RWP.14,FALSE,1.5629104,0.0912037,NA,NA
Cre01.g000050,RWP.14,FALSE,1.9702945,0.3410131,NA,NA
Cre01.g000050,RWP.14,FALSE,0.9607626,0.4324472,NA,NA


[1] 2898    7

#### 4. Export `gene_data` to `Gargouri_2015.txt`.

In [234]:
temp<-export_genes(df = gene_data, target_folder = target_folder)
temp[1:3,]
dim(temp)

gene_id,gene_name,candidate_gene,fold_difference,p_value,protein_id,protein_name
<chr>,<chr>,<lgl>,<dbl>,<dbl>,<lgl>,<lgl>
Cre01.g000050,RWP.14,FALSE,1.5629104,0.0912037,NA,NA
Cre01.g000050,RWP.14,FALSE,1.9702945,0.3410131,NA,NA
Cre01.g000050,RWP.14,FALSE,0.9607626,0.4324472,NA,NA


[1] 2898    7

## Goodenough_2014.xlsx
#### 1. Extract basic information
#### 2. Append basic information to `lipid_selection/data/intermediate_data_01/basic_info.txt`

In [236]:
#Basic information

file_name = "Goodenough_2014.xlsx"
genome_version = 4
database_source = "Augustus 10.2"
inclusion_criteria = c("fold difference greater than 2")
first_author = "Goodenough"
publication_year = 2014

basic_info <- data.frame(first_author, 
                         publication_year, 
                         genome_version, 
                         database_source, 
                         inclusion_criteria, 
                         stringsAsFactors = FALSE)
str(basic_info)

append_basic_info(basic_info, "../../intermediate_data_01/basic_info.txt")

'data.frame':	1 obs. of  5 variables:
 $ first_author      : chr "Goodenough"
 $ publication_year  : num 2014
 $ genome_version    : num 4
 $ database_source   : chr "Augustus 10.2"
 $ inclusion_criteria: chr "fold difference greater than 2"


In [237]:
#Check file type and number of excel sheets
sheets <- check_excel_data(file_name)  
print(length(sheets))
print(sheets)

[1] 24
 [1] "Table 1"    "Table 2"    "Table 3"    "Table 4"    "Table 5"   
 [6] "Table 6"    "Table 7"    "Table 8"    "Table 9"    "Table S1"  
[11] "Table S2"   "Table S3"   "Table S4"   "Table S5"   "Table S6"  
[16] "Table S7"   "Table S8"   "Table S9"   "Dataset S1" "Dataset S2"
[21] "Dataset S3" "Dataset S4" "Dataset S5" "Dataset S6"


#### 3. Extract candidate and non-candidate genes based on inclusion criteria.

**Inclusion criteria**: If fold-difference between 0 hour and *n* hours after N starvation >2, count as candidate gene. 
    
* Only use excel sheets "Table 2" to "Table 7"
* Exclude acetate boost data

In [238]:
require("readxl")
require("tidyverse")
require("tidyr")
require("dplyr")
require("stringr")
gene_data<- data.frame(matrix(nrow = 0, ncol = 4))
#Data manipulation: combine TF and TR gene expression data
#gene_data<- data.frame(matrix(nrow = 0, ncol = 10))
for (sheet in sheets[2:7]){
    
    #Import relevant columns, 
    #first 10 columns do not include acetate-boosted expression
    df <- read_excel(file_name, sheet = sheet)[,1:10] 
    colnames(df)<- c('name',df[2,2:10])
    
    #Assign gene id to each row, ignoring strain names in experiment
    gene_id <-NA
    for (row in 1:nrow(df)){
        if(TRUE %in% stringr::str_detect(df[row,1],"^Cre.+")){
            gene_id<- grep("^Cre.+", df[row,1], value = TRUE)
        }
    else{df[row,1] <- gene_id}
    }

    #Reshape dataframe to compare expression levels to time 0
    gene_data <- df %>% rename(control = "0 h") %>%
        gather(ends_with('h'), key = "time", value = "expression") %>%
        na.omit() %>% select(-log) %>%
        mutate(control = as.numeric(control),
              expression = as.numeric(expression)) %>%
        
        #Calculate fold difference relative to control time 0H
        mutate(fold_difference = control/expression,
        #assign candidate gene label based of fold difference in expression level
               candidate_gene = case_when(
            abs(log(fold_difference,2))>1  ~ TRUE,
            TRUE ~ FALSE)) %>%

    #add gene_data to working dataframe gene_data
        rbind(gene_data, .)
}


New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* ... and 12 more problems
New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* ... and 12 more problems
New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* ... and 12 more problems
New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* ... and 12 more problems
New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* ... and 12 more problems
New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* ... and 12 more problems


In [239]:
#Split "name" column to "gene_id" and "gene_name" columns in gene_data
gene_data<- gene_data %>% 
    separate(., name, sep = "\\s+", into=c("gene_id","gene_name")) %>%
    select(gene_id, gene_name, candidate_gene, fold_difference)
print(gene_data_temp[31,])

Warning message:
"Expected 2 pieces. Additional pieces discarded in 924 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...]."Warning message:
"Expected 2 pieces. Missing pieces filled with `NA` in 14 rows [523, 524, 533, 534, 543, 544, 553, 554, 563, 564, 573, 574, 583, 584]."

# A tibble: 1 x 3
  gene_id       gene_name candidate_gene
  <chr>         <chr>     <lgl>         
1 Cre03.g205050 DGTT4     TRUE          


In [240]:
#Export gene_data
temp<-export_genes(df = gene_data, target_folder = target_folder)
temp[1:3,]

gene_id,gene_name,candidate_gene,fold_difference,protein_id,protein_name
<chr>,<chr>,<lgl>,<dbl>,<lgl>,<lgl>
Cre03.g175400,PGI1,FALSE,0.8522727,NA,NA
Cre03.g175400,PGI1,FALSE,0.6838906,NA,NA
Cre13.g598750,GPM1b,FALSE,0.7904762,NA,NA


### Hemme_2014.xlsx

#### 1. Extract basic information
#### 2. Append basic information to `lipid_selection/data/intermediate_data_01/basic_info.txt`

In [241]:
file_name = "Hemme_2014.xlsx"
genome_version = NA
database_source = "Augustus 10.2"
inclusion_criteria = c("fold difference greater than 2", "p-value <0.05")
first_author = "Hemme"
publication_year = 2014

basic_info <- data.frame(first_author, 
                         publication_year, 
                         genome_version, 
                         database_source, 
                         inclusion_criteria, 
                         stringsAsFactors = FALSE)
str(basic_info)

append_basic_info(basic_info, "../../intermediate_data_01/basic_info.txt")

'data.frame':	2 obs. of  5 variables:
 $ first_author      : chr  "Hemme" "Hemme"
 $ publication_year  : num  2014 2014
 $ genome_version    : logi  NA NA
 $ database_source   : chr  "Augustus 10.2" "Augustus 10.2"
 $ inclusion_criteria: chr  "fold difference greater than 2" "p-value <0.05"


In [242]:
#Check file type and number of excel sheets
sheets <- check_excel_data(file_name)  
print(length(sheets))
print(sheets)

[1] 1
[1] "Protein data"


#### 3. Extract candidate and non-candidate genes based on inclusion criteria.

**Inclusion criteria**: When fold-difference between 0 hour and *n* hours after heat stress (HS) is statistically signficant (p-value <0.05), count as candidate gene. 
    
* Significance in expression is defined as >2 fold-difference in a time point relative to the control (time= 0 hours) within the HS treatment **(not recovery)**
* column `TP24HS_fold_change` is the linear fold difference of HS at time = 24 hours compared to time = 0 hour. 
* use p-value from `C1_significance`

In [253]:
require("readxl")
require("tidyverse")
require("dplyr")

#Data manipulation: combine TF and TR gene expression data
df <- import_messy_excel(file_name, sheet_name = "Protein data")
#colnames(df)

gene_data <- df %>% 
    
    #rename columns
    rename( gene_name = DisplayId, gene_id = Gene_identifier, fold_difference = TP24HS_fold_change, 
           p_value = C1_Significance ) %>%
    
    #select relevant columns
    select(gene_id, gene_name, fold_difference, p_value) %>%
    
    #assign candidate gene label based on inclusion criteria
    mutate( fold_difference = as.numeric(fold_difference),
           p_value = as.numeric(p_value),
           candidate_gene = case_when(
                p_value < 0.05 & abs(log(fold_difference,2))>1 ~ TRUE,
                TRUE ~ FALSE))

gene_data[1:3,]

New names:
* `` -> ...2
* `` -> ...3
* `` -> ...4
* `` -> ...5
* `` -> ...6
* ... and 45 more problems
Check if column names are correct.


gene_id,gene_name,fold_difference,p_value,candidate_gene
<chr>,<chr>,<dbl>,<dbl>,<lgl>
Cre17.g720250.t1,LHCB4,0.74,1.82e-01,FALSE
Cre16.g673650.t1,LHCB5,1.32,5.10e-10,FALSE
Cre02.g110750.t1,LHCB7,0.43,2.43e-01,FALSE


#### 4. Export `gene_data` to `Hemme_2014.txt`.

In [257]:
temp<- export_genes(df = gene_data, target_folder = target_folder)
temp[1:3,]
dim(temp)

gene_id,gene_name,fold_difference,p_value,candidate_gene,protein_id,protein_name
<chr>,<chr>,<dbl>,<dbl>,<lgl>,<lgl>,<lgl>
Cre17.g720250.t1,LHCB4,0.74,1.82e-01,FALSE,NA,NA
Cre16.g673650.t1,LHCB5,1.32,5.10e-10,FALSE,NA,NA
Cre02.g110750.t1,LHCB7,0.43,2.43e-01,FALSE,NA,NA


[1] 1985    7

### Juergens_2015.xls

#### 1. Extract basic information
#### 2. Append basic information to `lipid_selection/data/intermediate_data_01/basic_info.txt`

In [ ]:
#Basic information

file_name = "Juergens_2015.xls"
genome_version =  NA
database_source = c("Augustus 10.2","Phytozome 10.0")
inclusion_criteria = "fold difference greater than 2"
first_author = "Juergens"
publication_year = 2015

basic_info <- data.frame(first_author, 
                         publication_year, 
                         genome_version, 
                         database_source, 
                         inclusion_criteria, 
                         stringsAsFactors = FALSE)
str(basic_info)

append_basic_info(basic_info, "../../intermediate_data_01/basic_info.txt")

In [ ]:
#Check file type and number of excel sheets
sheets <- check_excel_data(file_name)  
print(length(sheets))

#### 3. Extract candidate and non-candidate genes based on inclusion criteria.

**Inclusion criteria**: If transcript log2 fold change and/or 

**Sheet 1: "Day 3" **